In [1]:
from __future__ import annotations

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

from helper import *

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Intro

# Load Data

In [3]:
# Reduced data to make it manageable for smaller systems
DATA_FILE: str = '../data/shakespeare_small.txt'

In [4]:
with open(DATA_FILE, 'r') as data_file:
    raw_text = data_file.read()

In [5]:
print(f'Number of characters: {len(raw_text):,}')

Number of characters: 50,085


# Character-Based Generation

## Encode Text into Integer Tokens

### Normalization

> - Skip?
> - lowercase?

In [6]:
def normalize_text(text: str) -> str:
    # Can be multiple actions to normalize text
    normalized_text = text.lower()
    return normalized_text

In [7]:
normalized_text = normalize_text(raw_text)

### Pretokenization

In [8]:
def pretokenize_text(text: str) -> list[str]:
    smaller_pieces = [char for char in text]
    return smaller_pieces

In [9]:
shakespeare_text = pretokenize_text(normalized_text)

### Tokenize

In [10]:
character_mapping = TokenMapping(shakespeare_text)

In [11]:
encoded = character_mapping.encode(shakespeare_text)

### Postprocessing

> Skip

## Prepare Dataset

In [12]:
n_tokens = character_mapping.n_tokens
dataset_size = len(encoded)

In [13]:
length = 32 # Number of characters
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [14]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [15]:
def generate_text(
    model,
    input_str: str,
    num_chars: int = 100,
    temperature: float = 1.0,
) -> str:
    model.eval()
    generated_text = input_str.lower()
    input_tensor = tokens_to_index_tensor(
        tokens=input_str.lower(),
        token_index_mapping=character_mapping.token2index,
    )
    
    with torch.no_grad():
        for _ in range(num_chars):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_char_idx = torch.multinomial(probabilities, 1).item()
            generated_text += character_mapping.index2token[next_char_idx]
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_char_idx]], dtype=torch.long),
                ],
                1,
            )
            
    return generated_text

## Train Model

In [16]:
PHRASE = 'To be or not to be'
epochs = 5

start = start_time()
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    gen_output = generate_text(model, PHRASE, 100)
    print(gen_output)

Epoch 1/5, Loss: 2.267284755661084
 6.2s (0 0%) 2.2403]
------------------------------------------------------------------------
to be or not to beinfeed, fases, fobven lat and reat's a marn's,
them your sil thusester the lent, we efirs and as i t
Epoch 2/5, Loss: 1.9004900568971237
 12.7s (1 20%) 1.7704]
------------------------------------------------------------------------
to be or not to be toonsed forth?

aufinius:
be asingiemnien:
thouve sy, ble make all you dome,
you beter: he enow
how
Epoch 3/5, Loss: 1.8204258402315572
 19.0s (2 40%) 1.9521]
------------------------------------------------------------------------
to be or not to be sthoun.

aupilg tory but blang.

fiurfor sust,
erciors,
the agiard
the cester.

hing the tor brojte
Epoch 4/5, Loss: 1.7804084676523177
 25.3s (3 60%) 1.5967]
------------------------------------------------------------------------
to be or not to be seren s
poodius lied bus i hey dow, be hange the pare hanger not i varcy the no,
nos it deyes sencr

## Generate Text

In [17]:
output = generate_text(model, 'To be or not to be', 100)
print(output)

to be or not to belies,
hese
in them,
thet'est shall shall you, wim, mepone. in't the made prace teliinicer hened yet,


# Token-Based Generation

## Encode Text into Integer Tokens

### Tokenize (Choose)

In [18]:
# Choose a pretrained tokenizer to use
xlmr_model_name = 'xlm-roberta-base'
bert_model_name = 'bert-base-cased'
bert_model_name_uncased = 'bert-base-uncased'

bert_uncased_tokenizer = AutoTokenizer.from_pretrained(
    bert_model_name_uncased,
)

In [19]:
shakespeare_text = (
    tokenize_text(
        tokenizer=bert_uncased_tokenizer,
        text=raw_text,
    )
)

In [20]:
token_mapping = TokenMapping(shakespeare_text)

In [21]:
encoded = token_mapping.encode(shakespeare_text)

## Prepare Dataset

In [22]:
n_tokens = token_mapping.n_tokens
dataset_size = len(encoded)

In [23]:
length = 16  # Tokens 
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [24]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [25]:
def generate_text(
    tokenizer,
    model,
    input_str: str,
    num_tokens: int = 100,
    temperature: float = 1.0,
) -> str:
    model.eval()
    tokenized_text = tokenize_text(tokenizer=tokenizer, text=input_str)
    input_tensor = tokens_to_index_tensor(
        tokens=tokenized_text,
        token_index_mapping=token_mapping.token2index,
    )

    generated_text = []
    with torch.no_grad():
        for _ in range(num_tokens):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_token_idx = torch.multinomial(probabilities, 1).item()
            generated_text.append(token_mapping.index2token[next_token_idx])
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_token_idx]], dtype=torch.long),
                ],
                1,
            )
    # Convert to text again
    output_ids = tokenizer.convert_tokens_to_ids(generated_text)
    output_str = input_str + ' ' + tokenizer.decode(output_ids)
    return output_str

## Train Model

In [26]:
epochs = 5

start = start_time()
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    output = generate_text(bert_uncased_tokenizer, model, 'To be or not to be', 30)
    print(output)


Epoch 1/5, Loss: 6.228423463167065
 2.7s (0 0%) 5.6661]
------------------------------------------------------------------------
To be or not to be now om our'advanced in when evenrily, conducted, give ; not but you on tell relieve do eleven equal part a, youeness corn
Epoch 2/5, Loss: 5.539322401775351
 5.4s (1 20%) 4.9891]
------------------------------------------------------------------------
To be or not to be have render i first within accuseus : so, to theia. brutus that and citizen : hat all come : your could any sick of hate
Epoch 3/5, Loss: 4.976599313916951
 8.2s (2 40%) 4.9341]
------------------------------------------------------------------------
To be or not to be a verdict ; their the whip of to us to death nothing a fairness we sons : yet, mean a wills budge good after by and your flatius
Epoch 4/5, Loss: 4.561570403059614
 10.9s (3 60%) 3.8148]
------------------------------------------------------------------------
To be or not to be one three than and they'soth, th

## Generate Text

In [27]:
output = generate_text(bert_uncased_tokenizer, model, 'To be or not to be', 30)
print(output)

To be or not to be the chestsric less flameh as he another :nc, sir. messenger : i fool appear like their for man ; i's hate, but


# Comparison Between Generation